In [36]:
import pandas as pd

job_ads = pd.read_json('Testfil_FINAL_2.json')

In [37]:
from nltk import word_tokenize
from nltk.text import Text
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
import string

# Load the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Define a list to store the sentiment scores
sentiment_scores = []

# Create a list of all the job ad descriptions
texts = [t.lower() for t in job_ads['description_text'].tolist()]

# Create a stop_words set
stop_words = set(stopwords.words('swedish'))

# Tokenize the descriptions and remove stopwords
tokens = [[w for w in word_tokenize(d) if not w in stop_words] for d in texts]

# Create an NLTK Text object from the tokens
text = Text([t for d in tokens for t in d])

# Define the keywords to search for
keywords = ['god', 'analytisk', 'förmåga']

# Define the number of characters to show before and after the keyword
window_size = 3

# Iterate through each keyword
for keyword in keywords:
    # Get the indices of all occurrences of the keyword
    indices = [i for i, w in enumerate(text) if w == keyword]

    # Define a list to store the KWIC for the keyword
    kwic = []

    # Generate the KWIC for each occurrence of the keyword
    for i in indices:
        left = ' '.join(text[i-window_size:i])
        right = ' '.join(text[i+1:i+window_size+1])
        #print(left, text[i], right)
        #print('-' * 50)
        kwic.append(left + ' ' + keyword + ' ' + right)
    
    # Define a list to store the sentiment scores for the KWIC
    keyword_sentiment_scores = []

    # Loop through each KWIC and get the sentiment score
    for k in kwic:
        # Remove punctuations and stopwords from the KWIC
        k = k.translate(str.maketrans('', '', string.punctuation))
        k = [word for word in k.split() if word not in stop_words]

        # Check if KWIC is not empty
        if k:
            # Get the sentiment score for the KWIC
            sentiment_score = sia.polarity_scores(' '.join(k))['compound']
            keyword_sentiment_scores.append(sentiment_score)

    # Calculate the mean sentiment score for the keyword
    if keyword_sentiment_scores:
        keyword_mean_sentiment_score = sum(keyword_sentiment_scores) / len(keyword_sentiment_scores)
        sentiment_scores.append(keyword_mean_sentiment_score)

        # Print the mean sentiment score for the keyword
        print(f"Mean sentiment score for '{keyword}': {keyword_mean_sentiment_score:.4f}")
    else:
        print(f"No sentiment score found for '{keyword}'")


Mean sentiment score for 'god': 0.2792
Mean sentiment score for 'analytisk': 0.0810
Mean sentiment score for 'förmåga': 0.1102


In [38]:
from collections import Counter
# Identify the frequently occurring keywords in the KWIC output
keywords = [w for w in text if w != keyword and w.isalpha()]
keywords_counter = Counter(keywords)
print('Frequently occurring keywords:')
for word, count in keywords_counter.most_common(10):
    print(f'{word}: {count}')

Frequently occurring keywords:
kommer: 11207
erfarenhet: 7553
arbeta: 7254
hos: 6544
söker: 6240
vill: 5708
samt: 5562
tjänsten: 5254
kunder: 5049
in: 4581


In [45]:
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import word_tokenize
from nltk.text import Text
from nltk.corpus import stopwords
from textblob import TextBlob

# Load the sentiment analyzers
sia = SentimentIntensityAnalyzer()

# Define a list to store the sentiment scores
sentiment_scores = []

# Create a list of all the job ad descriptions
texts = [t.lower() for t in job_ads['description_text'].tolist()]

# Create a stop_words set
stop_words = set(stopwords.words('swedish'))

# Tokenize the descriptions and remove stopwords
tokens = [[w for w in word_tokenize(d) if not w in stop_words] for d in texts]

# Create an NLTK Text object from the tokens
text = Text([t for d in tokens for t in d])

# Define the keywords to search for
keywords = ['stark',
'drivkraft',
'chef',
'analys',
'analytisk',
'driven',
'individer',
'beslut',
'kompetent',
'självständig']

# Define the number of characters to show before and after the keyword
window_size = 15

# Iterate through each keyword
for keyword in keywords:
    # Get the indices of all occurrences of the keyword
    indices = [i for i, w in enumerate(text) if w == keyword]

    # Define a list to store the KWIC for the keyword
    kwic = []

    # Generate the KWIC for each occurrence of the keyword
    for i in indices:
        left = ' '.join(text[i-window_size:i])
        right = ' '.join(text[i+1:i+window_size+1])
        kwic.append(left + ' ' + keyword + ' ' + right)

    # Calculate the sentiment score for the KWICs
    kwic_sentiment_scores = []
    for kwic_text in kwic:
        sia_scores = sia.polarity_scores(kwic_text)
        tb_scores = TextBlob(kwic_text).sentiment
        # Combine the sentiment scores from the two analyzers
        compound_score = sia_scores['compound'] + tb_scores.polarity
        kwic_sentiment_scores.append(compound_score)

    # Calculate the average sentiment score for the keyword
    avg_sentiment = sum(kwic_sentiment_scores) / len(kwic_sentiment_scores)
    sentiment_scores.append(avg_sentiment)
    print(f"Keyword: {keyword}, Sentiment score: {avg_sentiment:.2f}")

# Calculate the average sentiment score
avg_sentiment_score = sum(sentiment_scores) / len(sentiment_scores)
print("Average sentiment score:", avg_sentiment_score)


Keyword: stark, Sentiment score: -0.15
Keyword: drivkraft, Sentiment score: 0.06
Keyword: chef, Sentiment score: 0.03
Keyword: analys, Sentiment score: 0.03
Keyword: analytisk, Sentiment score: 0.13
Keyword: driven, Sentiment score: 0.15
Keyword: individer, Sentiment score: 0.11
Keyword: beslut, Sentiment score: 0.05
Keyword: kompetent, Sentiment score: 0.04
Keyword: självständig, Sentiment score: 0.11
Average sentiment score: 0.05688113563153881
